In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
import torch
from transformers import BertModel, AdamW

class ProteinBertClassifier(torch.nn.Module):
    def __init__(self, bert_model_name, num_labels):
        super().__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_output = outputs[0][:, 0, :] # get the CLS token output
        cls_output = self.classifier(cls_output)
        return cls_output

In [ ]:
from transformers import BertModel, BertTokenizer
import re

# bert_model_name = 'Rostlab/prot_bert' 
bert_model_name = "yarongef/DistilProtBert"

# Number of output labels (classes)
num_labels = 2

tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=False )
model = ProteinBertClassifier(bert_model_name, num_labels).to(device)

# Freezing the parameters of the BertModel
for param in model.bert.parameters():
    param.requires_grad = False

# Leaving the classifier unfrozen (can train)
for param in model.classifier.parameters():
    param.requires_grad = True

# Optimizer
optimizer = AdamW(model.classifier.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()